In [2]:
import tensorflow as tf

In [3]:
class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, db_dir, batch_size, input_shape, num_classes, 
                 shuffle=True, dir_classes=False):
        
        self.current_counter = 0
        self.label_dict = {}

        self.input_shape = input_shape
        self.batch_size = batch_size
        self.num_classes = num_classes
        self.shuffle = shuffle
        
        # load the data from the root directory
        self.data, self.labels = self.get_data(db_dir)
        self.indices = np.arange(len(self.data))
        self.on_epoch_end()


    def get_data(self, root_dir):
        """"
        Loads the paths to the images and their corresponding labels from the database directory
        """
        self.data = []
        self.labels = []

        path = self.db_dir

        for l in os.listdir(path):
          
          
        
          cd = None
          try:
            cd = Image.open(path + '/' + l)
            cd = np.asarray(cd)
          except:
            continue

          #self.data.append(path+'/'+l)
          #self.labels.append(1 if str(l[0]).islower() else 0)
          l_class = "".join(l.split('_')[:-1])
          if l_class not in self.label_dict.keys():
            self.label_dict[l_class] = self.current_counter
            self.current_counter += 1
          #self.labels
        
        """
            Second loop.
        """
        for l in os.listdir(path):
          cd = None
          try:
            cd = Image.open(path + '/' + l)
            cd = np.asarray(cd)
          except:
            continue

          self.data.append(path+'/'+l)
          l_class = "".join(l.split('_')[:-1])
#           self.labels.append(self.label_dict[l_class])
          self.labels.append(0 if l[0].islower() else 1)
    
        return self.data, self.labels

    def __len__(self):
        """
        Returns the number of batches per epoch: the total size of the dataset divided by the batch size
        """
        return int(np.floor(len(self.data) / self.batch_size))
        # TODO your code  here (size of dataset divided by the batch size)
        return 0
    
    def vec_to_tensor(self, v):
      return constant(v)
    
    """
        Preprocesses before outputting to .fit()
        This can mean: reshape, channel count check, mean subtraction and std. dev division.
    """
    def batch_to_train(self, batch_x, batch_y):
        x_train = [self.vec_to_tensor(x) for x in batch_x]
        y_train = [self.vec_to_tensor(x) for x in batch_y]

        #print(self.input_shape)
        x_train = [cv2.resize(np.array(i), (self.input_shape[0], self.input_shape[1])) for i in x_train]
        x_train2 = []
        y_train2 = []

        for i in range(len(x_train)):
          if (x_train[i].shape[0] == self.input_shape[0] and x_train[i].shape[1] == self.input_shape[1] and x_train[i].shape[2] == 3):
            x_train2.append(x_train[i])
            y_train2.append([y_train[i].numpy(), 0 if y_train[i].numpy() == 1 else 1])

        y_test = y_train2[:floor(1/5 * self.batch_size)]
        x_test = x_train2[:floor(1/5 * self.batch_size)]

        x_train = x_train2[ceil(1/5 * self.batch_size + 1):]
        y_train = y_train2[ceil(1/5 * self.batch_size + 1):]

        
        
        x_train = np.array(x_train)
        y_train = np.array(y_train)
        
#         mean = x_train.mean(axis=(0, 1, 2))
#         x_train[..., 0] = x_train[..., 0] - mean[0]
#         x_train[..., 1] = x_train[..., 1] - mean[1]
#         x_train[..., 2] = x_train[..., 2] - mean[2]
        
        x_test = np.array(x_test)
        y_test = np.array(y_test)
            
        return x_train, y_train

    def __getitem__(self, index):
        """"
        Generates a batch of data
        Note: computation intensivity
        """

        batch_indices = self.indices[index*self.batch_size : (index+1)*self.batch_size]
        
        batch_x = [] # TODO load the image(s) from batch_indices
        batch_y = [] # TODO load the corresponding labels of the images you loaded

        for i in batch_indices:
          ci = self.data[i]
          cl = self.labels[i]
            
          cd = Image.open(ci)
          cd = np.asarray(cd)

          if (len(cd.shape) != 3):
            continue
          # reshape?

          batch_x.append(cd)
          batch_y.append(cl)
        
         

        # optionally you can use: batch_y = tf.keras.utils.to_categorical(batch_y, num_classes=self.num_classes)
        return self.batch_to_train(batch_x, batch_y)

    def on_epoch_end(self):
        """"
        Called at the end of each epoch
        """
        # if required, shuffle your data after each epoch
        self.indices = np.arange(len(self.data))
        if self.shuffle:
            # TODO shuffle data
            # you might find np.random.shuffle useful here
            np.random.shuffle(self.indices)


In [ ]:
d = DataGenerator(
    db_dir='./dataset',
    batch_size=4,
    input_shape=(120, 120),
    dir_classes=True
)